In [ ]:
library(dplyr)
library(ggplot2)
library(ggpmisc)

In [ ]:
   # for stat_poly_eq()
# , rr.label, p.value.label can add
# 1) Compute median per donor (median age, median unique_SNV, cohort)
donor_medians <- Hiatt_novaf %>%
  group_by(donor) %>%
  summarise(
    median_unique_SNV = median(unique_SNVs, na.rm = TRUE),
    median_age = median(age, na.rm = TRUE),
    cohort = first(cohort),   # assumes cohort is constant per donor
    .groups = "drop"
  )

# quick check
print(head(donor_medians))

# Formula for lm
fm <- y ~ x

# 2) Plot A — overall regression on donor medians
p_overall <- ggplot() +
  # faint raw samples for context (optional)
  geom_point(data = Hiatt, aes(x = age, y = unique_SNVs),
             colour = "grey80", size = 1.2, alpha = 0.5) +
  # donor medians
  geom_point(data = donor_medians,
             aes(x = median_age, y = median_unique_SNV),
             shape = 23, size = 4, fill = "red", color = "black") +
  # regression fit computed on donor medians
  geom_smooth(data = donor_medians,
              aes(x = median_age, y = median_unique_SNV),
              method = "lm", se = TRUE, color = "steelblue", size = 0.8) +
  # equation / R^2 / p for regression fit on donor medians
  stat_poly_eq(
    data = donor_medians,
    aes(x = median_age, y = median_unique_SNV,
        label = after_stat(paste(eq.label, sep = "~~~"))),
    formula = fm,
    parse = TRUE,
    label.x.npc = "left",   # place at left
    label.y.npc = 0.95,
    size = 4
  ) +
  theme_minimal(base_size = 14) +
  labs(title = "Regression of median unique_SNV per donor (overall)",
       x = "Median age per donor",
       y = "Median unique_SNV per donor") +
  theme(legend.position = "none")

print(p_overall)


# 3) Plot B — faceted by cohort, regression done per facet using donor medians
# If some cohorts have very few donors, the equation may be unstable; you'll get warning if so.
p_facet <- ggplot() +
  geom_point(data = Hiatt, aes(x = age, y = unique_SNVs),
             colour = "grey90", size = 1.0, alpha = 0.5) +
  geom_point(data = donor_medians,
             aes(x = median_age, y = median_unique_SNV),
             shape = 23, size = 3.5, fill = "red", color = "black") +
  # add per-cohort regression lines using donor medians; grouped by cohort automatically via facet
  geom_smooth(data = donor_medians,
              aes(x = median_age, y = median_unique_SNV),
              method = "lm", se = TRUE, color = "steelblue", size = 0.8) +
  # per-facet equation annotation (stat_poly_eq will compute per-panel when data = donor_medians)
  stat_poly_eq(
    data = donor_medians,
    aes(x = median_age, y = median_unique_SNV,
        label = after_stat(paste(eq.label, rr.label, p.value.label, sep = "~~~"))),
    formula = fm,
    parse = TRUE,
    label.x.npc = 0.05,
    label.y.npc = 0.95,
    size = 3.6
  ) +
  facet_wrap(~ cohort, ncol = 2, scales = "fixed") +
  theme_minimal(base_size = 12) +
  labs(title = "Regression of donor medians, faceted by cohort",
       x = "Median age per donor",
       y = "Median unique_SNV per donor") +
  theme(legend.position = "none",
        strip.background = element_rect(fill = "grey95"))

print(p_facet)


# 1) Compute median per donor *per side*
donor_side_medians <- Hiatt %>%
  group_by(donor, side) %>%
  summarise(
    median_unique_SNV = median(unique_SNVs, na.rm = TRUE),
    median_age = median(age, na.rm = TRUE),
    .groups = "drop"
  )

# Optional check
print(donor_side_medians)

# formula
fm <- y ~ x

# 2) Plot: medians per donor per side, with regression lines & equations
p_side <- ggplot(donor_side_medians,
                 aes(x = median_age, y = median_unique_SNV, color = side)) +

  # median points
  geom_point(shape = 21, size = 3, fill = "white", stroke = 1.1) +

  # regression lines by side
  geom_smooth(method = "lm", se = TRUE) +

  # regression equations (separate per side automatically)
  stat_poly_eq(
    aes(
      label = after_stat(paste(eq.label, rr.label, p.value.label, sep = "~~~"))
    ),
    formula = fm,
    parse = TRUE,
    label.x.npc = 0.05,
    label.y.npc = c(0.90, 0.75),   # offsets so Left and Right labels don't overlap
    size = 4
  ) +

  theme_minimal(base_size = 14) +
  labs(
    title = "Median unique_SNV per donor by side (Right vs Left)",
    x = "Median age per donor",
    y = "Median unique_SNV per donor",
    color = "Side"
  )

print(p_side)
library(broom)
tidy(fit_interaction)
library(dplyr)

donor_side_medians %>%
  group_by(side) %>%
  do(tidy(lm(median_unique_SNV ~ median_age, data = .)))

donor_side_medians <- donor_side_medians %>%
  mutate(age_c = scale(median_age, scale = FALSE))

fit_interaction_centered <- lm(
  median_unique_SNV ~ age_c * side,
  data = donor_side_medians
)

summary(fit_interaction_centered)

library(dplyr)
library(ggplot2)
library(ggpmisc)
library(broom)

dat_lr <- Hiatt_novaf %>% filter(side %in% c("Left", "Right"))

# Ensure Left is reference so the interaction term is Easy to interpret
dat_lr$side <- relevel(factor(dat_lr$side), ref = "Left")

fit_all <- lm(unique_SNVs ~ age * side, data = dat_lr)
summary(fit_all)
tidy(fit_all)


GLMM and GLMM2

In [ ]:
library(lme4)
library(glmmTMB)
library(DHARMa)
library(ggeffects)
library(dplyr)
library(lmerTest)
library(car)
library(stringr)
library(corrr)
library(ggplot2)
model_poisson_log <- glmer(
  unique_SNVs ~ age + cohort + study + (1 | donor),
  family = poisson(link = "log"),
  data = Hiatt
)
summary(model_poisson_log)

overdisp_fun <- function(model) {
  rdf <- df.residual(model)
  rp <- residuals(model, type = "pearson")
  Pearson.chisq <- sum(rp^2)
  prat <- Pearson.chisq / rdf
  pval <- pchisq(Pearson.chisq, df = rdf, lower.tail = FALSE)
  c(chisq = Pearson.chisq, ratio = prat, rdf = rdf, p = pval)
}


overdisp_fun(model_poisson_log)

# dispersion ratio is HIGH, so I tried neg binom for a bit, and then
# changed to genpois,

# more stuff in Notion Nov 4 page... also Nov 10

## broadly choosing model
m1 <- glmmTMB(unique_SNVs ~ age + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df)
# M2 best for unique_SNVs but C.T is better so
m2 <- glmmTMB(unique_SNVs ~ age + sex + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df)
m3 <- glmmTMB(unique_SNVs ~ age + sex + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df)
m4 <- glmmTMB(unique_SNVs ~ age + sex + side + cohort + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df)
m5 <- glmmTMB(unique_SNVs ~ age + sex + side + cohort + study + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df)

## broadly choosing model
m <- glmmTMB(C.T ~ age, family = genpois(link = "log"), data=combined_df_nocarcinoma)
m0 <- glmmTMB(C.T ~ age + (1 | donor), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m0b <- glmmTMB(C.T ~ age + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m1 <- glmmTMB(C.T ~ age + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m1b <- glmmTMB(C.T ~ age + (1 | donor) + (1 | study) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m2 <- glmmTMB(C.T ~ age + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m3 <- glmmTMB(C.T ~ age + sex + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m4 <- glmmTMB(C.T ~ age + sex + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m5 <- glmmTMB(C.T ~ age + sex + side + cohort + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m6 <- glmmTMB(C.T ~ age + sex + side + cohort + study + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)

# Compare AIC
AIC(m, m0, m0b, m1, m1b, m2, m3, m4, m5, m6)

## broadly choosing model
m <- glmmTMB(unique_SNVs ~ age, family = genpois(link = "log"), data=combined_df_nocarcinoma)
m0 <- glmmTMB(unique_SNVs ~ age + (1 | donor), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m0b <- glmmTMB(unique_SNVs ~ age + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m1 <- glmmTMB(unique_SNVs ~ age + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m1b <- glmmTMB(unique_SNVs ~ age + (1 | donor) + (1 | study) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m2 <- glmmTMB(unique_SNVs ~ age + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m3 <- glmmTMB(unique_SNVs ~ age + sex + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m4 <- glmmTMB(unique_SNVs ~ age + sex + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m5 <- glmmTMB(unique_SNVs ~ age + sex + side + cohort + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)
m6 <- glmmTMB(unique_SNVs ~ age + sex + side + cohort + study + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=combined_df_nocarcinoma)

# Compare AIC
AIC(m, m0, m0b, m1, m1b, m2, m3, m4, m5, m6)

## broadly choosing model
m <- glmmTMB(unique_SNVs ~ age, family = genpois(link = "log"), data=Hiatt_df)
m0 <- glmmTMB(unique_SNVs ~ age + (1 | donor), family = genpois(link = "log"), data=Hiatt_df)
m0b <- glmmTMB(unique_SNVs ~ age + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m1 <- glmmTMB(unique_SNVs ~ age + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m2 <- glmmTMB(unique_SNVs ~ age + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m3 <- glmmTMB(unique_SNVs ~ age + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m4 <- glmmTMB(unique_SNVs ~ age + side + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m5 <- glmmTMB(unique_SNVs ~ age + side + cohort + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)
m6 <- glmmTMB(unique_SNVs ~ age + side + cohort + (1 | donor) + offset(log(depth)), family = genpois(link = "log"), data=Hiatt_df)

# Compare AIC
AIC(m, m0, m0b, m1, m2, m3, m4, m5, m6)

plot(simulateResiduals(m0))
plot(simulateResiduals(m1))
plot(simulateResiduals(m))

Type 1: sequential, matters which is listed first
Type 2: tests for each main effect after the other main effect, assuming no interaction
Type 3: tests for the presence of a main effect after the other main effect and interaction. This approach is therefore valid in the presence of significant interactions.

In [ ]:
anova(model_no_region, m_genpois)
AIC(model_no_region, m_genpois)
Anova(m_genpois, type = 3)
df <- Hiatt
predictors <- c("age", "sex", "side", "cohort", "study")
#df <- Hiatt_df# your dataset
#predictors <- c("age", "cohort", "region", "coverage")  # candidate predictors
offset_var <- "depth_scaled"  # offset variable
random_var <- "donor"  # random effect
# -------------------------
# 1. Sparse factor levels
# -------------------------
cat("\n=== Sparse Factor Levels ===\n")
for (v in predictors) {
  if (is.factor(df[[v]]) | is.character(df[[v]])) {
    counts <- table(df[[v]])
    sparse <- counts[counts < 5]  # fewer than 5 observations
    if (length(sparse) > 0) {
      cat("Predictor:", v, "- sparse levels:\n")
      print(sparse)
    }
  }
}

# -------------------------
# 2. Offset issues
# -------------------------
cat("\n=== Offset Variable Check ===\n")
if (any(df[[offset_var]] <= 0, na.rm = TRUE)) {
  cat("Warning:", offset_var, "contains zero or negative values.\n")
  print(summary(df[[offset_var]]))
} else {
  cat(offset_var, "looks good.\n")
}

# -------------------------
# 3. High collinearity for numeric predictors
# -------------------------
cat("\n=== Numeric Collinearity Check ===\n")
numeric_vars <- predictors[sapply(df[predictors], is.numeric)]
if (length(numeric_vars) > 1) {
  cor_mat <- cor(df[numeric_vars], use="pairwise.complete.obs")
  print(cor_mat)
  high_cor <- which(abs(cor_mat) > 0.8 & abs(cor_mat) < 1, arr.ind = TRUE)
  if (nrow(high_cor) > 0) {
    cat("High correlation detected:\n")
    apply(high_cor, 1, function(idx) {
      cat(numeric_vars[idx[1]], "<->", numeric_vars[idx[2]], "=", cor_mat[idx[1], idx[2]], "\n")
    })
  } else {
    cat("No high correlations among numeric predictors.\n")
  }
} else {
  cat("Not enough numeric predictors to check collinearity.\n")
}

# -------------------------
# 4. Extreme values / outliers
# -------------------------
cat("\n=== Extreme Values ===\n")
for (v in numeric_vars) {
  q <- quantile(df[[v]], probs=c(0.01,0.99), na.rm=TRUE)
  outliers <- df[[v]] < q[1] | df[[v]] > q[2]
  if (any(outliers)) {
    cat("Predictor:", v, "- extreme values detected at 1% or 99% quantiles\n")
    print(summary(df[[v]][outliers]))
  }
}

In [ ]:
# SETTINGS
# -------------------------

# Outcomes to model
responses <- c("C.T", "CpG", "C.G", "C.A", "unique_SNVs")

# Fixed effects selected from stepwise GLM
fixed_effects <- "age + region + cohort"

# Data
df <- Hiatt

# Storage
models <- list()
model_summaries <- list()
dispersion_results <- data.frame(
  Outcome = character(),
  AIC = numeric(),
  Dispersion = numeric(),
  Dispersion_p = numeric(),
  stringsAsFactors = FALSE
)

# -------------------------
# MODEL FITTING LOOP
# -------------------------
for (resp in responses) {

  cat("\nFitting model for", resp, "...\n")

  # Build formula
  formula_text <- paste0(resp, " ~ ", fixed_effects, " + (1 | donor)")
  formula <- as.formula(formula_text)

  # Fit model
  model <- try(glmmTMB(
    formula,
    family = nbinom1(link = "log"),
    data = df
  ), silent = TRUE)

  if (inherits(model, "try-error")) {
    cat("Model failed for", resp, "\n")
    next
  }

  # Store model
  models[[resp]] <- model

  # Extract fixed effects summary
  sum_model <- summary(model)
  fe <- sum_model$coefficients$cond
  fe_df <- data.frame(
    Outcome = resp,
    Predictor = rownames(fe),
    Estimate = fe[, "Estimate"],
    StdError = fe[, "Std. Error"],
    zValue = fe[, "z value"],
    pValue = fe[, "Pr(>|z|)"],
    RateRatio = exp(fe[, "Estimate"]),
    stringsAsFactors = FALSE
  )

  model_summaries[[resp]] <- fe_df

  # DHARMa residuals & dispersion
  sim_res <- simulateResiduals(model)
  disp_test <- testDispersion(sim_res)

  dispersion_results <- rbind(
    dispersion_results,
    data.frame(
      Outcome = resp,
      AIC = AIC(model),
      Dispersion = disp_test$statistic,
      Dispersion_p = disp_test$p.value,
      stringsAsFactors = FALSE
    )
  )

  cat("Dispersion test for", resp, ":\n")
  print(disp_test)
}

# -------------------------
# COMBINE ALL COEFFICIENTS
# -------------------------
all_coeffs <- do.call(rbind, model_summaries)

# -------------------------
# RESULTS
# -------------------------
# Models: models[["C.T"]], etc.
# Coefficients table: all_coeffs
# Dispersion & AIC: dispersion_results

# View top rows
head(all_coeffs)
dispersion_results

all_coeffs[all_coeffs$Outcome == "C.T", ]
plot(simulateResiduals(models[["C.T"]]))

# Combine all coefficients from previous workflow
# all_coeffs already has: Outcome, Predictor, Estimate, StdError, zValue, pValue, RateRatio

# Add significance stars
all_coeffs <- all_coeffs %>%
  mutate(
    Signif = case_when(
      pValue < 0.001 ~ "***",
      pValue < 0.01  ~ "**",
      pValue < 0.05  ~ "*",
      pValue < 0.1   ~ ".",
      TRUE           ~ ""
    )
  )

# Round numeric columns for readability
all_coeffs <- all_coeffs %>%
  mutate(
    Estimate = round(Estimate, 3),
    StdError = round(StdError, 3),
    zValue = round(zValue, 2),
    pValue = signif(pValue, 3),
    RateRatio = round(RateRatio, 3)
  )

all_coeffs <- as_tibble(all_coeffs)

# Reorder columns for reporting
report_table <- all_coeffs %>%
  dplyr::select(Outcome, Predictor, Estimate, StdError, zValue, pValue, RateRatio, Signif)

# View table
print(report_table)

In [ ]:
m1 <- glmmTMB(
  C.T ~ age + offset(log(depth_scaled)),
  family = compois(link = "log"),
  data = combined_df,
  control = glmmTMBControl(optCtrl = list(iter.max = 1e5, eval.max = 1e5))
)

m2 <- glmmTMB(
  C.T ~ age + side + offset(log(depth_scaled)),
  family = compois(link = "log"),
  data = combined_df,
  control = glmmTMBControl(optCtrl = list(iter.max = 1e5, eval.max = 1e5))
)

m3 <- glmmTMB(
  C.T ~ age + (1 | donor) + offset(log(depth_scaled)),
  family = compois(link = "log"),
  data = combined_df,
  control = glmmTMBControl(optCtrl = list(iter.max = 1e5, eval.max = 1e5))
)

m4 <- glmmTMB(
  C.T ~ age + side + (1 | donor) + offset(log(depth_scaled)),
  family = compois(link = "log"),
  data = combined_df,
  control = glmmTMBControl(optCtrl = list(iter.max = 1e5, eval.max = 1e5))
)

AIC(m1,m2,m3,m4)

#   df      AIC
#m1  3 5102.814
#m2  4 5093.840
#m3  4 4650.466
#m4  5 4640.317

In [ ]:
plot(res)
plot(res2)
plot(res3)
plot(res4)

GLMM2

In [ ]:
library(lme4)
library(glmmTMB)
library(DHARMa)
library(ggeffects)
library(dplyr)
library(lmerTest)
library(car)
library(stringr)
library(corrr)
library(ggplot2)

In [ ]:
Hiatt$depth_scaled <- Hiatt$depth / 1e6

m_pois <- glmmTMB(
  C.T ~ age + cohort + offset(log(depth_scaled)) + (1|donor),
  family = poisson(),
  data = Hiatt
)

# Quick overdispersion check
res <- residuals(m_pois, type = "pearson")
dispersion_ratio <- sum(res^2) / df.residual(m_pois)
dispersion_ratio


library(DHARMa)

m_nb <- glmmTMB(
  unique_SNVs ~ age + region*cohort + offset(log(depth_scaled)) + (1|donor),
  family = nbinom2(),
  data = Hiatt
)

summary(m_nb)
res_nb <- simulateResiduals(m_nb, n = 1000)
plot(res_nb)